In [1]:
pip install -q datasets trl peft bitsandbytes sentencepiece wandb huggingface_hub

Note: you may need to restart the kernel to use updated packages.


In [1]:
from huggingface_hub import notebook_login
notebook_login()

In [1]:
import os
import gc
import torch

import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, BitsAndBytesConfig
from datasets import load_dataset
from peft import LoraConfig, PeftModel, get_peft_model, prepare_model_for_kbit_training
from trl import DPOTrainer
import bitsandbytes as bnb
import wandb
wandb.login()
%env WANDB_PROJECT=hindi_dpo_test

model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
new_model = "Hindi-SentenceRetrieval-Tinyllama-1.1B"

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: ahm-rimer. Use `wandb login --relogin` to force relogin


env: WANDB_PROJECT=hindi_dpo_test


In [2]:
def chatml_format(example):
    # Format system
    if len(example['task']) > 0:
        message = {"role": "system", "content": example['task']}
        system = tokenizer.apply_chat_template([message], tokenize=False)
    else:
        system = ""

    # Format instruction
    message = {"role": "user", "content": example['query']}
    prompt = tokenizer.apply_chat_template([message], tokenize=False, add_generation_prompt=True)

    # Format chosen answer
    chosen = example['pos'] + "<|im_end|>\n"

    # Format rejected answer
    rejected = example['neg'] + "<|im_end|>\n"

    return {
        "prompt": system + prompt,
        "chosen": chosen,
        "rejected": rejected,
    }

# Load dataset
dataset = load_dataset("TokenBender/e5_FT_sentence_retrieval_task_Hindi_mini")['train']

# Save columns
original_columns = dataset.column_names

# Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"

# Format dataset
dataset = dataset.map(
    chatml_format,
    remove_columns=original_columns
)

In [10]:
dataset

Dataset({
    features: ['prompt', 'chosen', 'rejected'],
    num_rows: 6633
})

In [13]:
print(dataset[0])

{'prompt': '<|system|>\nप्रश्न के रूप में एक सेलिब्रिटी का नाम दिए जाने पर, साक्षात्कार और जीवनी पुनर्प्राप्त करें।</s>\n<|user|>\nलियोनार्डो डिकैप्रियो एक अमेरिकी अभिनेता, फिल्म निर्माता और पर्यावरण कार्यकर्ता हैं। उन्हें छह अकादमी पुरस्कारों, चार ब्रिटिश अकादमी फिल्म पुरस्कारों और नौ स्क्रीन एक्टर्स गिल्ड पुरस्कारों के लिए नामांकित किया गया है, जिनमें से प्रत्येक पुरस्कार में से एक और ग्यारह नामांकनों में से तीन गोल्डन ग्लोब पुरस्कार जीते हैं। डिकैप्रियो ने 1980 के दशक के अंत में टेलीविजन विज्ञापनों में दिखाई देकर अपने करियर की शुरुआत की। इसके बाद उन्होंने विभिन्न टेलीविजन श्रृंखलाओं में आवर्ती भूमिकाएँ निभाईं, जैसे कि सोप ओपेरा सांता बारबरा और सिटकॉम ग्रोइंग पेन्स। उन्होंने रॉबर्ट डी नीरो के साथ संस्मरण दिस बॉयज़ लाइफ के फिल्म रूपांतरण में अभिनय करने से पहले क्रिटर्स 3 में जोश के रूप में अभिनय करके अपने फिल्म करियर की शुरुआत की। डिकैप्रियो को नाटक वॉट्स ईटिंग गिल्बर्ट ग्रेप (1993) में उनकी सहायक भूमिका के लिए सराहा गया था, और जेम्स कैमरून के महाकाव्य रोमांस टाइटैनिक (1997) के साथ अं

In [3]:
# LoRA configuration
peft_config = LoraConfig(
    r=16,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=['k_proj', 'gate_proj', 'v_proj', 'up_proj', 'q_proj', 'o_proj', 'down_proj']
)

In [4]:
# Model to fine-tune
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    load_in_4bit=True
)
model.config.use_cache = False

# Reference model
ref_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    load_in_4bit=True
)

In [ ]:
# Training arguments
training_args = TrainingArguments(
    per_device_train_batch_size=,
    gradient_accumulation_steps=4,
    gradient_checkpointing=True,
    learning_rate=5e-5,
    lr_scheduler_type="cosine",
    num_train_epochs=3,
    save_strategy="no",
    logging_steps=1,
    output_dir=new_model,
    optim="paged_adamw_32bit",
    warmup_steps=100,
    bf16=True,
    report_to="wandb",
)

# Create DPO trainer
dpo_trainer = DPOTrainer(
    model,
    ref_model,
    args=training_args,
    train_dataset=dataset,
    tokenizer=tokenizer,
    peft_config=peft_config,
    beta=0.1,
    max_prompt_length=1024,
    max_length=1536,
)

# Fine-tune model with DPO
dpo_trainer.train()

/opt/conda/lib/python3.10/site-packages/trl/trainer/dpo_trainer.py:314: UserWarning: When using DPODataCollatorWithPadding, you should set `remove_unused_columns=False` in your TrainingArguments we have set it for you, but you should do it yourself in the future.
  warnings.warn(


Map:   0%|          | 0/6633 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (4632 > 2048). Running this sequence through the model will result in indexing errors
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


/opt/conda/lib/python3.10/site-packages/bitsandbytes/nn/modules.py:226: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(f'Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.')
Could not estimate the number of tokens of the input, floating-point operations will not be computed


Step,Training Loss
1,0.703000
2,0.698300
3,0.703000
4,0.675100
5,0.694200
6,0.675100
7,0.675300
8,0.658600
9,0.641100
10,0.634400
